In [1]:
def mod_mul_standard(a, b, p):
    result = (a * b) % p
    if result > (p>>1):
        result -= p
    return result

def mod_add(a, b, p):
    result = (a + b) 
    if result > (p>>1):
        result -= p
    if result < -(p>>1):
        result += p
    return result

def mod_sub(a, b, p):
    result = (a - b) 
    if result > (p>>1):
        result -= p
    if result < -(p>>1):
        result += p
    return result

def check_arrays(a,b,p):
    if a == b:
        print("exactly equal")
        return True
    print("not exactly equal")

    if len(a) != len(b):
        print("lengths not equal")
        return False


    for i in range(len(a)):
        if (a[i] - b[i]) % p != 0:
            print("element", i, "not equal even in modulus class")
            return False
    print("But equal in modulus class")
    return True

In [2]:
# LinearConv.
def Linear_Conv(f,g,p):
    if p != 0:
        
        n = len(f)
        result = [0]*(2*n-1)
        for i in range(n):
            for j in range(n):
                result[i+j] += (f[i]*g[j] % p)
        return result

    else:

        n = len(f)
        result = [0]*(2*n-1)
        for i in range(n):
            for j in range(n):
                result[i+j] += (f[i]*g[j])
        return result



def NegaCyclic_Conv(f,g,p):
    if p != 0:

        n = len(f)
        result = Linear_Conv(f,g,p)
        for i in range(n-1):
            result[i] -= result[i+n] % p
        return result[:n]

    else:

        n = len(f)
        result = Linear_Conv(f,g,p)
        for i in range(n-1):
            result[i] -= result[i+n] 
        return result[:n]

In [3]:
def Mu(b,q):
    mu = 2*int(((b << 31)/(2*q))+0.5)
    return mu

def Barrett_mul(a, b, p, mu_b):
    # Perform the multiplication
    z = (a * b) % (1 << 32)
    if z >= (1 << 31):
        z -= (1 << 32)  # Convert to signed 32-bit integer

    # Perform the multiplication for mu_b and get the high 32 bits
    temp = (a * (mu_b << 1)) % (1 << 64)
    t_high = (temp >> 32) % (1 << 32)
    if t_high >= (1 << 31):
        t_high -= (1 << 32)  # Convert to signed 32-bit integer
    
    result = z - t_high * p
    # Simulate 32-bit signed integer overflow for the result
    result %= (1 << 32)
    if result >= (1 << 31):
        result -= (1 << 32)  # Convert to signed 32-bit integer

    return result

def Barrett_red(A, N, V, i=25):
    # Step 1: Doubling multiplication, high half (sqdmulh equivalent)
    T0 = (A * V) >> (i + 1)
    
    # Step 2: Signed shift right with rounding (srshr equivalent)
    T1 = (T0 + (1 << (i - 2))) >> (i - 1)
    
    # Step 3: Subtract modulus (mls equivalent)
    z = A - (T1 * N)
    
    # Ensure result is in range -N/2 ≤ z < N/2
    if z >= N // 2:
        z -= N
    elif z < -N // 2:
        z += N
    
    return z


def number_order(temp, p):
    maximum = max([abs(x) for x in temp])

    degree = 0
    while True:
        if degree*(p>>1) < maximum < (degree+1)*(p>>1):
            return degree
        degree += 1


In [4]:
N = 101  # Example odd modulus
i = N.bit_length()  # Bit length of N
R = 2**31  # Radix for 32-bit signed numbers
V = (2**(i - 2) * R) // N  # Multiplier as described

A = 123456  # Example number to reduce

# Perform Barrett Reduction
z = Barrett_red(A, N, V, i)
print(f"Result of Barrett Reduction: z = {z}")

Result of Barrett Reduction: z = -517811870469


In [5]:
def CT_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    # Second layer
    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    return t0, t1, t2, t3

def GS_4pts(P, t0, t1, t2, t3, omega_1, omega2, omega3, mu_omega_1, mu_omega2, mu_omega3):
    # First layer
    temp0 = mod_add(t0, t1, P)
    t1 = mod_sub(t0, t1, P)
    t0 = temp0

    temp1 = mod_add(t2, t3, P)
    t3 = mod_sub(t2, t3, P)
    t2 = temp1

    t1 = Barrett_mul(t1, omega2, P, mu_omega2)
    t3 = Barrett_mul(t3, omega3, P, mu_omega3)

    # Second layer
    temp0 = mod_add(t0, t2, P)
    t2 = mod_sub(t0, t2, P)
    t0 = temp0

    temp1 = mod_add(t1, t3, P)
    t3 = mod_sub(t1, t3, P)
    t1 = temp1

    t2 = Barrett_mul(t2, omega_1, P, mu_omega_1)
    t3 = Barrett_mul(t3, omega_1, P, mu_omega_1)

    return t0, t1, t2, t3

In [6]:
def forward_10(temp,ptr,P,omegas,mu_omegas):
    n = 1 << 10
    ptrf = ptr
    ptrg = ptrf + n


    for depth in range(0, 10, 2):
        logn_top = 10 - depth
        n = 1 << logn_top
        hhn = n >> 2
        power_index = 1 << depth

        for j in range(0, 1 << depth):
            for i in range(0, hhn):
                temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = CT_4pts(P, temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
                temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = CT_4pts(P, temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i], omegas[power_index], omegas[power_index<<1], omegas[(power_index<<1)+1], mu_omegas[power_index], mu_omegas[power_index<<1], mu_omegas[(power_index<<1)+1])
            power_index += 1
    
def backward_10(temp,ptr,P,omegas,mu_omegas):
    n = 1 << 10
    ptrf = ptr
    ptrg = ptrf + n


    for depth in range(8, -1, -2):
        logn = 10 - depth
        n = 1 << logn
        hhn = n >> 2
        power_index = 2*(1 << depth)-1

        for j in range(0, 1 << depth):
            for i in range(0, hhn):
                temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P, temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
                temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P, temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
            power_index -= 1

    

In [7]:
from data import *

logn_top = 10
n = 1 << logn_top
temp = [0] *  (8 * n)

temp[0:n] = f[0:n]
temp[n:2*n] = g[0:n]

ptrf = 0
ptrg = n

In [8]:
forward_10(temp,ptrf,P[0],omegas,mu_omegas)


In [9]:
print(temp)

[-435656, -5400622, 14551170, -14628658, -10278786, -3879386, 11016177, 15598448, 11740741, 9070733, -14620289, -717763, -12857682, -12499730, 8954476, -8410983, 4504662, -10847729, 13728358, -6014042, -10551078, -5842633, -11905405, -15704585, 10391451, 6978326, -3955142, 253091, 14565644, 3993063, -8742101, -16478465, -4558249, 10764973, -5699010, -257190, 7613289, 278392, 2168999, -1336145, -1546345, -7864933, 7527473, -8671020, 1258819, -14560521, -10996907, 9879745, -2352077, -12542855, -12787472, -1062464, 8648262, -13193259, 8329229, 5823247, 11335916, -2941838, 8585768, -16368172, 2754043, 4583696, -8098344, 12306646, -9691065, -5836568, -6777714, 10872593, -15650728, 9635981, -3984683, -3573324, 1072972, 9501936, -8470761, -6729738, 9883717, -6963115, -9277451, 14967589, -5910749, 10458373, -8323217, 5891588, -16504879, 10637871, -4245469, 15325376, -15321262, -12046611, 13872834, 1526107, -6467106, 13136344, -6467131, 8474103, -2630112, 1903282, -167874, -15228352, -4881872, 

In [10]:
h = [-435656, -5400622, 14551170, -14628658, -10278786, -3879386, 11016177, 15598448, -21809596, 9070733, -14620289, -717763, -12857682, -12499730, -24595861, -8410983, -29045675, -10847729, 13728358, -6014042, -10551078, -5842633, -11905405, -15704585, -23158886, 6978326, -3955142, 253091, -18984693, 3993063, -8742101, 17071872, -4558249, 10764973, -5699010, -257190, -25937048, 278392, 2168999, -1336145, -1546345, -7864933, 7527473, 24879317, 1258819, -14560521, -10996907, 9879745, -2352077, -12542855, -12787472, -1062464, 8648262, 20357078, 8329229, 5823247, -22214421, -2941838, 8585768, -16368172, 2754043, 4583696, -8098344, 12306646, -9691065, -5836568, -6777714, 10872593, -15650728, 9635981, -3984683, -3573324, 1072972, 9501936, -8470761, -6729738, 9883717, -6963115, -9277451, 14967589, -5910749, 10458373, -8323217, 5891588, -16504879, 10637871, -4245469, 15325376, -15321262, -12046611, -19677503, 1526107, -6467106, 13136344, -6467131, 42024440, -2630112, 1903282, -167874, -15228352, -38432209, -9507007, 12379096, 6236416, -10101876, 8278352, -13340363, -2393732, -8840237, 10314919, 1440403, 24739217, -37486363, 14811804, 14429241, 15396140, -17945080, 12290008, 9660694, 2099572, -35316455, 15131218, -1593218, -2492188, 3282655, 1135925, 13664808, 13357738, -36103, 644030, -6694771, -16228727, -2392191, 22689179, 8815054, -7913338, -10408871, 9792002, -4823303, -3603277, -27737277, -4667889, -1156059, 536548, -15601956, -9860868, 12478750, 20989562, 4773427, -2995299, -3724493, 4620912, -2248473, -11080519, -15896904, 31138075, -35340195, -11867429, -16258604, -15001473, 6173612, 25577424, -16050425, 6804501, -2948186, 10145282, -5183373, 21087144, 5877047, -10842661, 12872305, -12315157, -20892607, 8393015, -2727147, 18443272, -16722417, -14181444, -10579080, -61837, -16621760, -8703567, -5056344, 20165987, -34426294, -10359424, -14596976, -768155, 2644994, 6311289, -12670115, 23327682, 3823308, -5911934, -4183777, -803088, -20842656, 3528038, -10005409, 3872333, -10106528, 1248869, -9108744, 3282356, 13219405, -12565321, 4186952, 9142429, -51858949, 8993462, -9947285, -8345211, 6196177, 16839003, -16682193, -14306600, -20985307, -9726725, 4924044, -16391746, -38197966, -4922022, 14564861, -5988085, 7409591, 16619280, -16441257, -15427426, -7977071, 18266963, -9522707, 15051363, 4140593, -2929517, -11232147, -2152800, 5559250, 17057738, -9361209, 36260774, -14763585, -16712217, 13266355, 4422449, 13101580, 11318886, 10769169, 9687265, 16069076, -1810434, -16707840, 5085193, 15292505, -13232507, 11259924, 16553595, 11772281, 3232219, 8193771, 5162033, 9917149, 16233766, -3979613, -2155460, -14069919, -8347502, -16567756, -846676, -2663514, -6435764, 449392, 19634672, 12749518, 14644182, -4227817, 10101221, -7493496, 14130143, 9885539, -787223, -22249293, -8081021, -12455487, 12999625, 2038371, -15495061, 10495414, -12708868, -29397565, -25879672, -2558411, 21892511, -15306222, -1443454, -1042323, 14335179, 7699683, -12578875, -9706079, 15279108, -9501122, 10255488, 1222948, 26138861, -18683116, 12324103, 8131539, 16253550, -9328327, 2616760, -15995823, 16287990, -15806987, -7972881, -21763868, 10379982, -19547738, 8142978, -11017232, 10574001, -22151013, -12443120, -22265041, 10046583, -10998916, -806650, -7415661, 17975175, -16306159, 11424550, -10061111, 8744218, -3046438, 9099850, 8748858, 30989094, -16529019, -13775636, -11379397, 12754978, 8921219, 3730667, 14106961, 36051553, 13493717, 14541092, 15190743, 17234389, -2742779, 30698263, 7316865, 30046582, -10101659, 1982745, -19705485, 12478787, 3891062, 5012114, -11426627, 2801052, -21014922, -3040318, -2270885, 4645935, -8432875, 13373836, -5552266, 17328603, -308301, 16739363, -3441979, 27144547, -5721944, 4870440, -15135229, 9607180, 5156407, -6456999, 1307172, 2718297, -14268852, -4778984, 11061220, 49405733, 3660955, 820215, -2758979, 22399495, -26997739, -16386323, -8108636, 8758954, 16639401, -5149921, -8217551, -7583656, 12772158, 6995190, 13415179, 29338845, 6400343, 8594482, 4870189, 7825170, -29938280, -3879280, 11846386, 1079406, -19690920, -15518255, 738393, 30734501, -19341410, 12327166, 4913716, 782060, -36105283, -6261622, 8796604, 17710716, 7768583, 14134451, 10627342, 11924182, 801638, 23076524, -11448222, 13111293, -15132898, 9886700, 2981030, 20673581, -19081009, -5232141, 11704096, -7550504, 6588249, 926407, -24542234, -7337034, -12777255, 7759020, -2353342, 3344193, -7892290, 22709424, 8181003, 6780259, -3222143, 26717727, -2384361, 13000513, -15387348, -7481315, 16650131, 11573831, 754583, 3178201, 6960294, 9078963, -13970503, -10132189, -9722692, 45548093, -15160747, 16001723, 5238665, 8242424, -18481897, 5834015, -3987010, 15211901, -14539836, -5378431, 3219288, 12950588, 7915947, 14946157, -13476019, 11905850, -28192207, 5282430, -4195566, 9585424, -15079488, 10234861, 7117678, -5376072, -15025850, 5745513, -10341725, -2012873, -17545601, 1145249, -8123626, 9227817, -4336059, -7500767, 10287179, -1803191, -14237748, -2970619, 13160487, 16601341, -21071211, -15495485, -21972184, -6221464, -20372857, -4399691, -7827014, 5069480, -3295290, 637001, -13516170, -6820654, -29603542, -7260402, -12308071, 15528351, -24881852, -8769578, -11032094, 7565359, -4047580, -847055, -21632259, -2596235, -15807038, -6828438, -18305370, -213980, -13911131, -7019878, 16619038, 9016632, 15513960, 14896818, -7763780, 6409783, -13703985, 1168856, -10263381, 43292640, -41767735, -26384194, 12154880, -5484140, -16290158, 6563773, -10919364, -5604674, -7288623, -9043825, -23987424, 1217408, 12316599, 15071926, -8402302, 5658519, -2466917, 16454412, 1492115, 1275619, -14439993, -14960903, 15122317, 7090511, 6390242, -12687884, 500635, 32210829, -2008584, -1107303, -22161158, -3881860, -19863594, -1694115, 16377693, -8257801, -16954448, -3796652, -6658545, 23005843, -9706529, -2746394, -5590397, 14629123, 6883735, -2601715, 12493010, 233798, -4867606, 17770262, -1202463, 5482619, -5333112, 693074, -10367857, 178155, -24974598, -6504872, -3369858, 2984138, -14636246, 8230293, -19377027, 2459743, 2948015, 10267153, -9769942, -5732633, -8648491, 9875091, 11946550, 38017946, 383606, 15082422, -15323305, -2136432, -11969632, 8446616, 9750673, 8993027, -8317093, 799616, -12446295, -2829431, 8777612, 23014492, 1077220, -2056760, 439566, 4580895, -27344114, -596640, 6727903, 7828399, 5176801, 5740678, 6745252, 15557256, -7813437, -6827288, -8485543, -14977987, 12286631, 1911541, -12703685, 8436438, 7117969, 14676442, -28534418, 873232, 13015813, 43848015, -18290505, 11677300, -8796302, -14047944, -25091979, 14464040, -3666815, 14735848, 4195885, 9754321, 11717181, 11542575, -19935832, -8059656, 110216, -1771718, -13119466, -2523915, -2338698, 19361886, 2065705, -2029095, 9998136, 28196670, -14459159, 6840835, 12399339, -560853, 12670793, 15447724, -9343588, -207836, -36415189, -11349573, -9079911, 3048641, -6292443, 27064437, -5204998, -9611884, 7164188, 14376077, 738270, 10091920, 16626785, 9934411, -12376035, 35648344, -7025657, 3996719, -13272849, -6288013, 13381529, 14897892, 13124990, -1496730, -1424926, -1255479, -14571089, -9735819, -21681884, 2675924, -3038459, 9881250, -5485718, 19929512, -2551176, -14421978, 7766414, 10405315, 15859319, -12900805, 13879912, 18208564, 7686923, 8421382, -14334922, 5397778, 4059233, 76977, -32875820, -211150, -9429453, 17712101, -14915374, 7997973, -3506651, -16395061, -30448875, 2470891, -9459097, 1900014, 13850434, 7226600, 8328981, 9739901, 15143882, 16414996, 7721093, -6382407, 2793402, 13612746, -4260477, 23884269, -13891167, 16152169, 9636024, 34805942, -14294284, -2185675, -9878172, 44275479, -16193597, -11549468, -14692808, 1060852, 8175796, -4608926, 16000599, 15642339, -9925091, 6691124, 1443759, 22712035, 16703397, -15195177, 12591396, 3307162, -5699751, 13350430, 5000975, 19591423, 4067281, 15095708, -9059336, 7807577, 217166, 5570642, 14270410, 9885830, -11438622, 12936464, -11600966, 32646027, -22139634, -6871668, -6426211, -2327501, -17238398, -14621826, 2923166, -3129204, 16410264, 1017242, 12669260, -859402, -6077468, 10553368, 2936244, 12549215, -20985832, 12995541, -5192323, -13271701, -15644552, -8143758, -6698942, -5885809, -5597136, 9313603, 7521129, 3677289, -16350056, 16059071, 14152827, 18005997, 7091347, 23994615, -1824571, -1054819, 8182162, -14204578, 15607770, -609094, 4765500, 11242678, -3502194, -127132, -24139704, -1342046, 14502703, 4117331, -6945942, -10201850, -12585011, 4848760, -15584708, 12250204, 14312723, -16633627, -22438427, 7431983, -3052195, 13098430, 12942443, 10975475, 747798, 14852981, 9380332, 14742571, -11043097, 9076651, 9579768, 11342178, 9889500, 10894667, -8122652, -316232, -6868548, 25471296, -3801217, -1059117, -5279103, -1027860, 15181411, -1162717, -5883512, 5529758, -2691429, 12295293, -7058665, 4281626, 222488, 10432790, -15024783, 13413606, -19258670, 1178104, -12819299, 4445740, -8256573, -8185641, 11173504, 5667388, -14944064, -11121692, -25323874, -7023814, -282913, -14837343, 12166909, 34476703, -21669060, 14375921, -4783042, -2519198, -9532915, 690536, -2682304, 2485759, -11683110, 15261213, 2350934, 26102750, -17966945, 8771533, -13639524, 169549, -12933477, -12899968, 14698112, 31460952, -21852212, -6048912, -3269503, -8072023, 7189128, 13120516, -9289926, -6870562, 10867862, -11899230, 525054, 30928052, 15346441, 15033537, -16701147, -783132, -10307692, -2142223, -13813910, -13267589, -9762182, -481915, -27193826, -2547806, 9016343, -5558821, -13344873, 10829120, -7915807, 6276990, -10273125, 1091932, -16248638, -4597431, 4631655, 12457989, -12335503, -12927013, -17364217, -11355283, -13285025, -11380313, 4573559, 31640251, 718106, -11704956, 16546556, -171280, -1027920, -15837522, 15394152, 8093802, -7156971, 23076605, -1292329, 7494976, 13016266, -4913456, 11094395, -9820537, -14404325, -14478183, 7557462, 31353808, -9272030, 3818178, -7029970, 19059641, -29058997, -5675829, 5388787, 9982403, 4843078, 11619800, 10439544, 22569756, 9659427, 10060321, 9113260, 32917896, -10504673, -5396069, 7587680, -7843230, 2099360, 28828070, 13365083, -10215085, -9076640, 17791712, 12677734, 6911502, 5103074, -915144, 14483873, 18120155] 
check_arrays(temp[0:1024],h,P[0])

not exactly equal
But equal in modulus class


True

In [11]:
print("""
temp: |--- NTT(f) under P[0] ---|--- NTT(g) under P[0] ---|--->
""")



temp: |--- NTT(f) under P[0] ---|--- NTT(g) under P[0] ---|--->



In [12]:
hn = n >> 1
ptrNf = 2*n
ptrNg = 2*n + hn
print(ptrf, ptrg,ptrNf, ptrNg)

0 1024 2048 2560


In [13]:
for i in range(hn):
    temp[ptrNf + i] = mod_mul_standard( temp[ptrf + 2*i] , temp[ptrf + 2*i + 1],P[0])
    temp[ptrNg + i] = mod_mul_standard( temp[ptrg + 2*i] , temp[ptrg + 2*i + 1],P[0])

In [14]:
print(temp[ptrNf:ptrNf+hn])

[-4655104, 16156743, -2227192, -174659, 3161240, -5463690, -1469057, -1690740, 8670825, -4440857, 16297834, -7453023, 5956977, -2989190, 11917852, 14914922, -12979220, 14809381, 3311987, 14491542, 3308396, 3237897, -8126544, -10285886, 10026299, -13867479, -2738507, 5941392, 2618326, 9276251, 12532971, 4036181, 13166283, 9958215, -10815688, 1675514, 1315895, -12862485, 12686275, 13669617, 13674493, 5858582, -9372751, -11834995, -11333223, -7831557, -12319621, -732473, -345836, 9335259, -1613194, -11895936, 13611108, 9497779, -1864851, -7999652, 3296245, -15526544, 10079417, -3315100, -2044484, -16502292, -1673979, 14682771, -1031549, 11263915, 13088649, -16663669, -9716321, -15258472, -11472828, -2513876, 706564, 8338603, 4046584, -3174367, 2393646, -948760, -7572651, 10363561, -5509393, 4160369, 14177722, 2894467, -2231227, 12989353, 13101853, 10613958, 12971260, 14099134, -15171699, -16711465, 16770041, 16171858, 15859546, -2244633, -14118076, 9054174, 328126, -10778227, 7913579, 744

In [15]:
n = hn
hn = n >> 1
ptrf = ptrNf
ptrg = ptrNg
ptrNf = ptrg + hn
ptrNg = ptrNf + hn

print(ptrf, ptrg, ptrNf, ptrNg)


2048 2560 2816 3072


In [16]:
for i in range(hn):
    temp[ptrNf + i] = mod_mul_standard( temp[ptrf + 2*i] , temp[ptrf + 2*i + 1],P[0])
    temp[ptrNg + i] = mod_mul_standard( temp[ptrg + 2*i] , temp[ptrg + 2*i + 1],P[0])

In [17]:
print(temp[ptrNf:ptrNf+hn])


[14802130, 16520350, 13615370, -5116604, -4370440, -8852444, 3331087, -12680244, -5396347, -4935103, -8067181, -13557970, -16483136, -5700561, -14040269, -13436314, 6439402, 8813211, -4387293, -4794427, -106187, -11491745, 2841114, -9537798, 13197312, -4681046, 9509494, 1684802, 1448696, 15086765, -16634590, -8932979, 6656853, -11514819, -9704191, -11838026, 15559859, 7714188, -6817767, 9277046, 12687665, -12036028, -4911377, -5964430, 8445437, 11005881, 15231169, -6840072, 5428190, -8388758, -1774326, 14294446, -12206594, 13859514, -14104019, 987910, 10493315, -6046160, 10810131, -33579, 10662786, 1353119, -13324299, -11048389, 13702124, -11526222, 12094914, 9046493, -14524318, 4380718, 1067604, -3430585, -3838500, -5990508, 12557619, -10064716, 16216509, 6877892, -9771641, 4421507, -2574160, 1455647, -3184782, -11392850, 11276873, -10202082, 13106562, 15658289, -2235709, -2589761, -5588966, -9965610, 6831886, -15847039, 13987267, -12209598, 2964647, -9320043, 13610908, 262518, -14043

In [18]:
ptrf = ptrNf
ptrg = ptrNg

depth = 8
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1


In [19]:
depth = 6
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [20]:
depth = 4
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [21]:

print(temp[ptrNf:ptrNf+hn])
check_arrays(temp[ptrNf:ptrNf+hn],h,P[0])

[-14447933, -16139198, -9378066, 7208941, 15621598, -12818212, -11213390, -13909729, -14705713, -13149457, -9286364, -8181020, 4086208, -11338153, -10671509, 682103, 15942662, 17274074, -2446189, 13697775, 16444553, -11081035, 12121114, -7403883, -13343474, 29650972, 11577378, -6396512, 9363903, -725189, -4428162, 12221954, 14087764, 9959092, 18270922, 22189178, 23645370, 2750121, 7926063, 20811806, 27448500, 21198867, 18521686, 5330889, 13790698, 23303077, 334851, 22438888, 17232593, 16033078, 26612101, 28427787, 8238827, 2151022, 24376424, 26542374, 8879851, 29930670, 13272135, 9515870, 23044040, 29519191, 30167754, 20083366, 16737074, 9894278, -7335276, -10331632, 4939129, 978821, -8347301, 3568248, -14312200, -7235117, 10427247, 13422387, 13855173, 9275874, -10268343, -15409515, 13496082, -4536448, -2549890, -16474057, 3781266, -2038492, 493401, 16100669, -10688843, 10381720, 15702095, 3781154, -8503630, 9260750, 16682624, 17069677, 1048153, 18495025, 30642657, 11286478, 1808666, 2

False

In [22]:
depth = 2
logn = 10 - depth
n = 1 << logn
hhn = n >> 2
power_index = 2*(1 << (depth-2))-1

for j in range(0, 1 << depth):
    for i in range(0, hhn):
        temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
        temp[ptrg + j*n + i], temp[ptrg + j*n + hhn + i], temp[ptrg + j*n + 2*hhn + i], temp[ptrg + j*n + 3*hhn + i] = GS_4pts(P[0], temp[ptrf + j*n + i], temp[ptrf + j*n + hhn + i], temp[ptrf + j*n + 2*hhn + i], temp[ptrf + j*n + 3*hhn + i], -omegas[power_index], -omegas[(power_index<<1)+1], -omegas[power_index<<1], -(mu_omegas[power_index]-2), -(mu_omegas[(power_index<<1)+1]-2), -(mu_omegas[power_index<<1]-2))
    power_index -= 1

In [23]:
print(temp[ptrNf:ptrNf+hn])
print(hn)



[-4536064, 12803322, -14734860, 6708213, 16747259, 2053370, -10368019, 15600631, -1738758, 2024696, 15305462, -9284367, 932598, 197620, -13925139, 967153, -13718280, -15201295, 3042800, -4487959, -6556683, 10794488, -1975565, 12185582, 14338036, -6469137, 7885550, 3082731, 9765871, -1571854, 11448557, 7178986, 7545075, -1222416, -9176340, 10847472, 15828199, 12809202, -11342352, 12145139, 4493806, -10107920, -9249548, 6374126, -1569810, -8745997, 9290483, -8703508, -395797, -2602001, 5403633, 2699754, -14976788, 7127019, -9452557, 9275119, 16260839, -2545685, 9909481, 4007145, 8578791, 3280873, 6773230, 7231720, -1385497, -5656598, 16114668, 11210730, 8262885, 13090534, -8232725, 11873259, 16466664, 4999920, 24770028, -7686168, 12524521, -4075034, 1106668, 8037106, 4508133, 3785192, 7230188, -5910548, -7509523, 27519720, -1456658, -15135766, 3557345, 230889, -506646, 370410, -4399638, 52961, 15513063, 17654255, -13304344, 4017373, -1521686, 20625645, -9628953, -6863642, 1527019, -32312

In [24]:
inv2 = 16775169
inv4 = mod_mul_standard(inv2, inv2, P[0])
mu_inv4 = Mu(inv4, P[0])

inv16 = mod_mul_standard(inv4, inv4, P[0])
mu_inv16 = Mu(inv16, P[0])
inv64 = mod_mul_standard(inv16, inv4, P[0])
mu_inv64 = Mu(inv64, P[0])
inv256 = mod_mul_standard(inv64, inv4, P[0])
mu_inv256 = Mu(inv256, P[0])
print(inv256)
print(mu_inv256)
inv1024 = mod_mul_standard(inv256, inv4, P[0])
mu_inv1024 = Mu(inv1024, P[0])




for i in range(ptrNf,ptrNf+hn):
    temp[i] = mod_mul_standard(temp[i], inv256, P[0])

print(temp[ptrNf:ptrNf+hn])


-131056
-8388606
[-17719, 836349, 1515114, 1467820, 720699, 794357, 2449564, 1240444, 779544, 1056357, 1370347, 1929573, 1314203, 1573444, 2435669, 1969618, 994861, 1906460, 2108782, 2996757, 1416004, 1090614, 1696011, 2406608, 1628680, 2202682, 2389811, 2764218, 2266100, 1828644, 2534785, 2911275, 1733201, 2092121, 2585275, 2139269, 3338229, 1884820, 2052590, 1751170, 2376562, 2057412, 1536541, 2383907, 2352876, 1669564, 1740019, 2587122, 2750630, 2217788, 1986948, 2893778, 2562617, 2780016, 1666804, 2264183, 3339919, 2742232, 3052997, 3029941, 3309911, 3027104, 2385466, 3173593, 3270988, 2861136, 2684068, 2927024, 3570789, 3458591, 2720017, 2798556, 3209667, 2116427, 2717878, 3115320, 3063212, 3391538, 2625443, 1866179, 3556122, 3160130, 2649363, 2598032, 2460730, 3252843, 2353318, 2824108, 4076632, 3015190, 2881253, 2884679, 2866046, 4062943, 3336998, 2296914, 3093374, 4602653, 2877288, 2570633, 3238787, 3380645, 2758141, 2477442, 3188338, 3443529, 2511792, 2155552, 2635721, 3734293

In [25]:
f_star = [(-1)**i * f[i] for i in range(len(f))]
print(f)
print(f_star)

[0, 0, 4, 1, 4, 1, 0, 1, 3, 2, 0, 3, 6, 0, 3, 2, 3, 0, 0, 1, 0, 4, 0, 2, 0, 2, 3, 3, 5, 2, 0, 1, 2, 2, 0, 1, 2, 0, 1, 4, 0, 1, 1, 3, 0, 1, 1, 2, 4, 1, 2, 0, 4, 0, 1, 6, 0, 4, 1, 0, 1, 2, 3, 0, 2, 0, 3, 2, 2, 1, 2, 2, 3, 3, 3, 0, 0, 3, 2, 1, 1, 0, 0, 4, 4, 0, 4, 0, 0, 1, 2, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 5, 4, 4, 2, 7, 0, 1, 1, 4, 0, 2, 3, 1, 3, 2, 2, 0, 1, 5, 0, 0, 2, 3, 2, 2, 2, 3, 2, 1, 1, 0, 1, 0, 1, 3, 1, 4, 0, 4, 3, 3, 0, 1, 3, 1, 3, 5, 3, 0, 1, 1, 2, 2, 1, 0, 4, 5, 4, 3, 5, 1, 1, 2, 1, 1, 2, 3, 0, 4, 0, 0, 4, 1, 2, 0, 1, 3, 0, 4, 3, 0, 2, 0, 0, 3, 1, 6, 0, 5, 7, 0, 2, 2, 1, 0, 2, 0, 5, 1, 4, 0, 0, 1, 1, 2, 1, 3, 4, 5, 1, 0, 0, 1, 4, 2, 0, 0, 2, 4, 3, 3, 3, 5, 1, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 1, 0, 2, 3, 1, 0, 0, 4, 1, 3, 0, 1, 0, 1, 4, 0, 1, 2, 1, 2, 3, 0, 3, 5, 4, 4, 1, 2, 1, 1, 1, 1, 5, 5, 1, 2, 5, 1, 4, 1, 0, 0, 2, 6, 0, 4, 4, 0, 0, 0, 7, 4, 4, 1, 0, 0, 0, 0, 0, 1, 4, 2, 4, 1, 6, 2, 9, 2, 2, 3, 2, 1, 0, 0, 2, 2, 0, 1, 4, 3, 3, 3, 1, 1, 0, 2, 0, 0, 0, 4, 1, 7, 0, 0, 1, 2, 3, 4, 

In [26]:
f = NegaCyclic_Conv(f,f_star,0)
f = f[::2]
print(f)
f_star = [(-1)**i * f[i] for i in range(len(f))]

f = NegaCyclic_Conv(f,f_star,0)
print(f[::2])

[-132, -179, 34, -133, -217, -161, -201, -155, -171, -160, -128, -98, -55, -53, -139, -123, -82, -53, -127, -177, -182, -136, -132, -134, -182, -175, -101, -149, -187, -200, -372, -99, 7, -41, 57, -1, 68, -2, -60, -5, -97, -156, -176, 182, -139, -111, -262, 27, -70, -382, -382, -112, -182, -240, -142, -287, -510, -227, -232, -377, -223, -106, -341, -262, -656, -268, -15, -24, -98, -140, -111, -308, -582, -393, -243, -147, -146, -165, -256, -247, -299, -141, -159, -184, -37, -283, -332, -199, -308, -390, -358, -168, -189, -325, -187, -257, -92, -356, -320, -64, -166, -138, -138, -256, -348, -246, -138, -336, -183, -183, -163, 27, -396, -69, -109, -106, -316, -57, -329, -161, -332, -349, -318, -78, -435, -160, -164, -355, -186, -269, -437, 6, -322, -247, -199, -63, -283, -117, -96, -463, -333, -287, -208, -161, -266, -219, -440, -97, -352, -268, -258, -242, -381, -397, -119, -362, -346, -293, -325, -255, -275, -513, -537, -502, -520, -123, 46, -432, -482, 88, -324, -292, -178, -213, -167

In [27]:
h = [-17719, 836349, 1515114, 1467820, 720699, 794357, 2449564, 1240444, 779544, 1056357, 1370347, 1929573, 1314203, 1573444, 2435669, 1969618, 994861, 1906460, 2108782, 2996757, 1416004, 1090614, 1696011, 2406608, 1628680, 2202682, 2389811, 2764218, 2266100, 1828644, 2534785, 2911275, 1733201, 2092121, 2585275, 2139269, 3338229, 1884820, 2052590, 1751170, 2376562, 2057412, 1536541, 2383907, 2352876, 1669564, 1740019, 2587122, 2750630, 2217788, 1986948, 2893778, 2562617, 2780016, 1666804, 2264183, 3339919, 2742232, 3052997, 3029941, 3309911, 3027104, 2385466, 3173593, 3270988, 2861136, 2684068, 2927024, 3570789, 3458591, 2720017, 2798556, 3209667, 2116427, 2717878, 3115320, 3063212, 3391538, 2625443, 1866179, 3556122, 3160130, 2649363, 2598032, 2460730, 3252843, 2353318, 2824108, 4076632, 3015190, 2881253, 2884679, 2866046, 4062943, 3336998, 2296914, 3093374, 4602653, 2877288, 2570633, 3238787, 3380645, 2758141, 2477442, 3188338, 3443529, 2511792, 2155552, 2635721, 3734293, 3309044, 2233831, 3286863, 2989702, 2714723, 2160829, 3409047, 3392354, 2474599, 3085299, 3209789, 3608029, 2671357, 2005876, 2043761, 2985364, 1337391, 2546892, 1935780, 3069797, 2288379, 2148967, 2424805, 2506329, 2762636, 2684835, 2392092, 2973440, 2764708, 2264403, 2233302, 3151992, 2741510, 3124294, 2513083, 3022090, 2893781, 2444895, 2279554, 2905770, 2850937, 2884069, 1805614, 2246756, 3112220, 2234405, 1985640, 2048931, 1804958, 2443776, 1727477, 2225309, 2109374, 3033923, 940356, 2507534, 2871501, 2130000, 1815224, 1922307, 2061637, 2193398, 2182954, 1568596, 2327156, 1918351, 1269169, 585030, 2185748, 2640746, 1891577, 1923815, 2021646, 2445980, 1227923, 2112225, 1466955, 1795572, 1833918, 1029111, 1219834, 1537292, 1345645, 452582, 1403651, 1444080, 1235327, 307227, 1332081, 746660, 1184612, -219477, 199030, 1080963, 674275, 13074, -146798, 32992, 102430, -503846, -398730, 310659, 101756, -370969, 48726, 484146, 718063, 39901, -670114, 1164766, 67477, -1378281, -516671, -305768, 341209, -402040, 169006, 189187, 193073, -745304, -279230, -649750, 884175, -1039518, -795839, -978375, -800126, -1356988, -2206944, -1238863, -1519336, -1580224, -1562125, -1241262, -1366808, -658558, -1387790, -1357414, -536251, -512231, -881104, -1837127, -568034, -1455885, -766112, -2134820] 
print(check_arrays(temp[ptrNf:ptrNf+hn],h,P[0]))


exactly equal
True


In [28]:
inv2s = [16775169,16770049,16769025,16766977]

inv4s = [mod_mul_standard(inv2s[i], inv2s[i], P[i]) for i in range(4)]
print(inv4s)
mu_inv4s = [Mu(inv4s[i], P[i]) for i in range(4)]
print(mu_inv4s)

[-8387584, -8385024, -8384512, -8383488]
[-536870894, -536870894, -536870894, -536870894]
